In [1]:
import re

In [2]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    text = text.replace('\ufeff', '') # убираем первый невидимый символ
    text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [3]:
text

'Вы  лучший ответ на проблемы которые возникли в понедельникДумайте позитивно и верьте в свою способность достигать отличных результатовЕсли вы смогли в понедельник подняться с постели значит вы супер герой'

In [5]:
len(text)

205

In [6]:
num_characters = 34 #33 буквы + пробел

In [10]:
import keras
from tensorflow.keras.preprocessing.text import Tokenizer

In [11]:
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

In [12]:
tokenizer.fit_on_texts(text)

In [13]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'т': 3, 'е': 4, 'и': 5, 'в': 6, 'н': 7, 'с': 8, 'л': 9, 'п': 10, 'ь': 11, 'ы': 12, 'р': 13, 'а': 14, 'д': 15, 'у': 16, 'к': 17, 'з': 18, 'ч': 19, 'й': 20, 'м': 21, 'г': 22, 'б': 23, 'я': 24, 'ш': 25, 'ю': 26, 'х': 27}


In [14]:
inp_chars = 6 #
data = tokenizer.texts_to_matrix(text)

In [15]:
data[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [16]:
n = data.shape[0]-inp_chars
n  #размер обучающего множества

199

In [17]:
import numpy as np

In [18]:
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа

In [19]:
len(X)

199

In [20]:
X[0]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [21]:
Y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
from keras.layers import *
from keras.models import Sequential

In [23]:
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 500)            │       267,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 34)             │        17,034 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 284,534 (1.09 MB)

 Trainable params: 284,534 (1.09 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9826 - loss: 0.1114  
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9582 - loss: 0.3066 
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9724 - loss: 0.1509 
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9734 - loss: 0.0793 
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9911 - loss: 0.0415 
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9914 - loss: 0.0411 
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9901 - loss: 0.0459 
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9963 - loss: 0.0305 
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9919 - loss: 0.0380 
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9910 - loss: 0.0489 
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9710 - loss: 0.0975 
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9887 - lo

In [26]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding

    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str

In [28]:
res = buildPhrase("утренн")
print(res)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━

# Слова


In [29]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
    texts = f.read()
    texts = texts.replace('\ufeff', '') # убираем первый невидимый символ

In [30]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
                       lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])

In [31]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])

[('вы', 3), ('лучший', 1), ('ответ', 1), ('на', 1), ('проблемы', 1), ('которые', 1), ('возникли', 1), ('в', 3), ('понедельник', 2), ('думайте', 1)]


In [32]:
data = tokenizer.texts_to_sequences([texts])

In [33]:
data

[[1,
  4,
  5,
  6,
  7,
  8,
  9,
  2,
  3,
  10,
  11,
  12,
  13,
  2,
  14,
  15,
  16,
  17,
  18,
  19,
  1,
  20,
  2,
  3,
  21,
  22,
  23,
  24,
  1,
  25,
  26]]

In [34]:
len(data[0])

31

In [35]:
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(31, 1000)


In [36]:
inp_words = 3
n = res.shape[0]-inp_words

In [37]:
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]

In [38]:
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │       144,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1000)           │       129,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 273,512 (1.04 MB)

 Trainable params: 273,512 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

In [39]:
history = model.fit(X, Y, batch_size=32, epochs=19)

Epoch 1/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 915ms/step - accuracy: 0.0000e+00 - loss: 6.9184
Epoch 2/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0000e+00 - loss: 6.8926
Epoch 3/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0357 - loss: 6.8668
Epoch 4/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.1786 - loss: 6.8406
Epoch 5/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3929 - loss: 6.8139
Epoch 6/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5357 - loss: 6.7864
Epoch 7/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7143 - loss: 6.7578
Epoch 8/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8571 - loss: 6.7278
Epoch 9/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8929 - loss: 6.6961
Epoch 10/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9286 - loss: 6.6623
Epoch 11/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9286 - loss: 6.6261
Epoch 12/19
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9286 - loss:

In [40]:
def buildPhrase(texts, str_len = 20):
  res = texts
  dataa = tokenizer.texts_to_sequences([texts])[0]
  for i in range(str_len):
    x = keras.utils.to_categorical(dataa[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
    inp = x.reshape(1, inp_words, maxWordsCount)
    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    indx = pred.argmax(axis=1)[0]
    data.append(indx)
    res += " " + tokenizer.index_word[indx] # дописываем строку
  return res

In [41]:
for i in range(1):
    x = keras.utils.to_categorical(data[i: i+inp_words], num_classes=maxWordsCount)

In [42]:
x.shape

(1, 31, 1000)

In [1]:
res = buildPhrase("позитив добавляет годы")
print(res)

NameError: name 'buildPhrase' is not defined